In [2]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import json
import os
import mayavi
from mayavi import mlab
mlab.init_notebook('x3d', 600, 600) # x3d or png

Notebook initialized with x3d backend.


# Notes
Translation tests measure the most basic feature of an interface tracking method. In this test, a hollow square, two tilted hollow square and a hollow circle are advected in constant velocity fields, periodic boudnary conditions are set at each of the . he material area and the shape should remain unchanged through the translation process.

## Model setup

The initial setup is

| Parameter  | value                         | 
|------------|-------------------------------|
| domain     | [0,2] $\times$ [0,2]          | 
| grids      | 100 $\times$ 100 $\times$ 100       |
| $h$        | 0.02                                |
| shape      | D = 0.4, d = 0.2                    |
| angle      | $\theta_1$ = 26.57, $\theta_2$ = 45 |
| velocity   | $\vec u = [1,1,1]^T$          |
| Time step  | $k=0.005$         |
| $n_{step}$ | $400$         |

# Setup

## Generate input file

In [3]:
f=h5py.File('input.h5','w') # open hdf5 file

In [4]:
# Initialize array
nx = 100
ny = 100
nz = 100
X,Y,Z = np.meshgrid(np.arange(ny),np.arange(nz),np.arange(nx))
phi = np.zeros([nz,ny,nx])
cx = np.zeros([nz,ny,nx])
cy = np.zeros([nz,ny,nx])
cz = np.zeros([nz,ny,nx])
u = np.zeros([nz,ny,nx])
v = np.zeros([nz,ny,nx]) 
w = np.zeros([nz,ny,nx]) 

In [ ]:
# hollow cube
phi[]

u = np.zeros([nz,ny,nx])+ 1.0
v = np.zeros([nz,ny,nx]) + 1.0
w = np.zeros([nz,ny,nx]) + 1.0
phi[5:10,5:10,5:10] = 1.0
cx[5:10,5:10,5:10] = 0.5
cy[5:10,5:10,5:10] = 0.5
cz[5:10,5:10,5:10] = 0.5

In [ ]:
f3.create_group('phi')
grp = f3['phi']
grp.create_dataset("init", data=phi)
f3.create_group('u')
grp = f3['u']
grp.create_dataset("init", data=u)
f3.create_group('v')
grp = f3['v']
grp.create_dataset("init", data=v)
f3.create_group('w')
grp = f3['w']
f3.create_group('cx')
grp.create_dataset("init", data=w)
grp = f3['cx']
f3.create_group('cy')
grp.create_dataset("init", data=cx)
grp = f3['cy']
f3.create_group('cz')
grp.create_dataset("init", data=cy)
grp = f3['cz']
grp.create_dataset("init", data=cz)